In [ ]:
%pip install transformers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch,gc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import transformers
import json
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from transformers import (
  BertTokenizerFast,
  AutoModelForSequenceClassification,
)
import logging
from torch import cuda
from sklearn.metrics import confusion_matrix,classification_report
import os

# define label list

In [3]:
df = pd.read_csv("/content/drive/MyDrive/文字探勘/final/training.csv")

In [4]:
# encoding the label for classifier's target
df['label'] = df['Category'] + ' ' + df['SubCategory']
df['label'], categories = pd.factorize(df['label'])
# categories的順序代表label從0-53

In [5]:
categories = [x.replace(' / ', '/') for x in categories]

In [6]:
categories

['男生衣著 T恤',
 '女生衣著 長褲',
 '女生衣著 裙裝',
 '女生衣著 洋裝',
 '女生衣著 細肩帶/繞頸背心',
 '女生衣著 牛仔褲',
 '女生衣著 毛衣/針織',
 '女生衣著 T恤',
 '女生衣著 襯衫',
 '女生衣著 吊帶褲/吊帶裙',
 '女生衣著 短褲',
 '女生衣著 冬季外套/大衣',
 '女生衣著 其他外套',
 '女生衣著 夾克',
 '女生衣著 西裝',
 '男生衣著 襯衫',
 '男生衣著 長褲',
 '男生衣著 背心',
 '男生衣著 外套',
 '女生衣著 斗篷',
 '女生衣著 其他',
 '女生衣著 內衣',
 '女生衣著 絲襪',
 '女生衣著 內褲',
 '女生衣著 襪子',
 '女生衣著 緊身褲/內搭褲',
 '男生衣著 襪子',
 '女生衣著 塑身衣',
 '男生衣著 大學T',
 '女生衣著 帽T',
 '女生衣著 哺乳內衣',
 '女生衣著 褲裙',
 '女生衣著 個人套裝',
 '女生衣著 大學T',
 '女生衣著 背心外套',
 '女生衣著 連身長褲',
 '男生衣著 其他',
 '男生衣著 內衣褲',
 '男生衣著 冬季外套/大衣',
 '女生衣著 成套內衣褲',
 '男生衣著 牛仔褲',
 '男生衣著 短褲',
 '男生衣著 工作褲',
 '男生衣著 帽T',
 '女生衣著 居家服',
 '男生衣著 縮口褲',
 '男生衣著 Polo衫',
 '女生衣著 Polo衫',
 '男生衣著 衛生衣',
 '男生衣著 套裝',
 '女生衣著 上衣',
 '男生衣著 坦克',
 '女生衣著 套裝',
 'Mask Mask']

# Evaluate on Jambo dataset

In [7]:
month = np.arange(start=1, stop=13, step=1)
month_arr = [f'{i:02}' for i in month]
month_arr

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [8]:
head = "/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_"
tail = ".xlsx"
Dataset_list = [head + x + tail for x in month_arr]
Dataset_list

['/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_01.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_02.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_03.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_04.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_05.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_06.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_07.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_08.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_09.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_10.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_11.xlsx',
 '/content/drive/MyDrive/文字探勘/final/Jumbo_Dataset/商品資料-2022_12.xlsx']

## Basic cleaning

In [ ]:
def cleaning(df):
    # rename the dataframe
    df.rename(columns={'name': 'ProductName'}, inplace=True)
    # 清除括號內的文字
    df["ProductName"] = df["ProductName"].replace(r'\([^)]*\)',"",regex=True)
    # 清除特殊符號
    special_symbols = r'[.．＂/<>:《》+\-=#$%&()~*@＃＄％＆＇\(\)\[\]\{\}（）＊＋－～／：\＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏★→─]+'
    df["ProductName"] = df["ProductName"].replace(special_symbols,"",regex=True)
    # 清除任何由英文及數字組成的字串
    df["ProductName"] = df["ProductName"].replace(r'[A-Z]+[0-9]+',"",regex=True)
    # 清除空格
    df["ProductName"] = df["ProductName"].replace(r'\s',"",regex=True)
    # drop na
    df["ProductName"] = df["ProductName"].dropna()

    return df

## Evaluating

In [10]:
# using gpu for training
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
# load model and tokenizer and define length of the text sequence
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [12]:
MAX_LENGTH = 64

In [13]:
def tokenization(Pname):
  input_ids = []
  attention_masks = []

  for name in Pname:
    # print(name)
    encoded_dict = tokenizer.encode_plus(
        name,
        add_special_tokens = True,
        max_length = MAX_LENGTH,
        padding = "max_length",
        truncation = True,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt',
    )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask
    attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  return input_ids,attention_masks


In [14]:
def evaluate(eval_dataloader):
    category = []
    sub_category = []
    with torch.no_grad():
        for batch_num, batch in enumerate(eval_dataloader):
            input_ids_eval,attention_masks_eval = batch
            input_ids_eval,attention_masks_eval = input_ids_eval.to(device),attention_masks_eval.to(device)
            logits = model(input_ids_eval,attention_mask = attention_masks_eval)
            predicts = np.argmax(logits.logits.cpu().numpy(),axis = 1)
            for label in predicts:
                tmp_categories = categories[label].split()
                category.append(tmp_categories[0])
                sub_category.append(tmp_categories[1])

    return category,sub_category


In [ ]:
# Evaluation
model = torch.load('/content/drive/MyDrive/文字探勘/final/models/model.pt')
model.eval()
for idx,dataset in enumerate(Dataset_list):
    print(f"evaluating month {month_arr[idx]}......")
    # read data by month
    jambo_df = pd.read_excel(dataset)
    # basic cleaning
    jambo_df = jambo_df[(jambo_df["category"] == "['女生衣著']") | (jambo_df["category"] == "['男生衣著']")]
    jambo_df = cleaning(jambo_df)
    # eval
    eval_data = jambo_df["ProductName"].to_numpy()
    input_ids,attention_masks = tokenization(eval_data)
    eval_dataset = TensorDataset(input_ids,attention_masks)
    eval_dataloader = DataLoader(
        eval_dataset,
        batch_size = 64,
        shuffle = False
    )
    category,sub_category = evaluate(eval_dataloader)
    jambo_df["new_Category"] = category
    jambo_df["SubCategory"] = sub_category

    jambo_df = jambo_df[(jambo_df["new_Category"] == "男生衣著") | (jambo_df["new_Category"] == "女生衣著")]
    location = "/content/drive/MyDrive/文字探勘/final/Dataset_Clothes/" + month_arr[idx] + "_origin.csv"
    jambo_df.to_csv(location)

    gc.collect()
    torch.cuda.empty_cache()

evaluating month 01......
evaluating month 02......
evaluating month 03......
